# Blackjack AR

In [ ]:
#cartas de poker
class Card:
    def __init__(self, suit, value, image):
        self.suit = suit
        self.value = value
        self.image = image
        self.inGame = False

In [ ]:
def init_cartas():
    for suit in suits:
        for value in values:
            card = Card(suit, value, f'cartas/{value}_of_{suit}.png')
            objects.append(card)

suits = ['clubs', 'diamonds', 'heards', 'spades']
values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace']
objects = []

init_cartas()

In [ ]:
import cv2
import numpy as np
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

#Capturar video desde la cámara
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
   
    # Mostrar el frame con el escudo, bufanda y marcas de colores
    cv2.imshow("Detección de Rostros con Bufanda y Marcas", frame)

    # Salir si se presiona escape
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()